In [ ]:
!pip install flask
!pip install pyngrok flask-ngrok

In [ ]:
from pyngrok import ngrok
ngrok.set_auth_token("TEXT2OBJMODEL_NGROK")

In [ ]:
from flask import Flask, request, send_file, jsonify
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from pyngrok import ngrok
import re


app = Flask(__name__)

# 1. 모델 로딩 (Colab에서 직접)
tokenizer = AutoTokenizer.from_pretrained("Zhengyi/LLaMA-Mesh")
model = AutoModelForCausalLM.from_pretrained(
    "Zhengyi/LLaMA-Mesh",
    torch_dtype=torch.float16,
    device_map="auto",
    low_cpu_mem_usage=True
)
model.eval()

# 정규식 파서 함수
def extract_obj_mesh(result: str) -> str:
    print(f"서버 생성본{result}")
    match = re.search(r"(?:```obj|'''obj)(.*)", result, re.DOTALL)
    if match:
        mesh_data = match.group(1).strip()
        # 마지막에 ''' 또는 ```로 끝나면 제거
        mesh_data = re.sub(r"(?:```|''')\s*$", "", mesh_data).strip()
        return mesh_data
    else:
        print(f"정규화 실패")
        return ""  # 메시 추출 실패 시 빈 문자열 반환



@app.route('/', methods=['GET'])
def health_check():
    return "success", 200

@app.route('/generate', methods=['POST'])
def generate_obj():
    data = request.get_json()
    if not data or 'object' not in data:
        return jsonify({"error": "Missing 'object' in request body"}), 400

    object_name = data['object']
    prompt = f"""
      Generate a 3D model of a {object_name}
      """


    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, padding=False).to(model.device)
    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_new_tokens=3000,
            do_sample=True,
            temperature=0.5,
            pad_token_id=tokenizer.eos_token_id
        )
    result = tokenizer.decode(output[0], skip_special_tokens=True)

    # 정규식으로 .obj 메시만 추출
    mesh = extract_obj_mesh(result)

    if not mesh:
        return jsonify({"error": "Failed to extract .obj mesh data"}), 500

    return jsonify({"mesh": mesh})

if __name__ == '__main__':
    public_url = ngrok.connect(5000).public_url
    print(f"Public URL: {public_url}")
    app.run(host='0.0.0.0', port=5000)


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Public URL: https://8343-34-71-148-149.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
